In [26]:
import numpy as np
import pandas as pd
import nltk
import sklearn as sk
import re # regex
import matplotlib

In [27]:
news_df = pd.read_csv('../datasets/news-20k-comments.csv')
politics_df = pd.read_csv("../datasets/politics-20k-comments.csv")
politicaldiscussion_df = pd.read_csv('../datasets/politicaldiscussion-20k-comments.csv')

raw_df = pd.concat([news_df, politics_df, politicaldiscussion_df]) # merge all three subreddit datas
raw_df.head()

,created_utc,ups,subreddit_id,link_id,name,score_hidden,author_flair_css_class,author_flair_text,subreddit,id,...,downs,archived,author,score,retrieved_on,body,distinguished,edited,controversiality,parent_id
0,1430438402,-11.0,t5_2qh3l,t3_34f1lq,t1_cqug92b,0.0,NaN,NaN,news,cqug92b,...,0.0,0.0,hogsucker,-11.0,1.432703e+09,1-She got to be a bigwig at Google by sleeping...,NaN,0.0,0.0,t1_cqu4t11
1,1430438407,1.0,t5_2qh3l,t3_34exjb,t1_cqug96h,0.0,NaN,NaN,news,cqug96h,...,0.0,0.0,flal4,1.0,1.432703e+09,For those about to lynch this guy [here](http:...,NaN,0.0,1.0,t1_cqudz0p
2,1430438439,4.0,t5_2qh3l,t3_34f10p,t1_cqug9tk,0.0,NaN,NaN,news,cqug9tk,...,0.0,0.0,HitachinoBia,4.0,1.432703e+09,It feels like black people are the most racist...,NaN,0.0,1.0,t1_cqufsip
3,1430438448,0.0,t5_2qh3l,t3_34cvvg,t1_cquga1l,0.0,NaN,NaN,news,cquga1l,...,0.0,0.0,[deleted],0.0,1.432703e+09,[deleted],NaN,0.0,0.0,t3_34cvvg
4,1430438449,-10.0,t5_2qh3l,t3_34e7eo,t1_cquga1v,0.0,NaN,NaN,news,cquga1v,...,0.0,0.0,Cultiststeve,-10.0,1.432703e+09,Its because otherwise thats all that would app...,NaN,0.0,0.0,t1_cqudxkr


In [28]:
all_comments = raw_df.filter(['created_utc', 'subreddit', 'body'])

In [29]:
#filter out deleted comments
comments = all_comments[all_comments['body'] != "[deleted]"]

In [30]:
# clean out any urls and and brackets, parenthesis and hyphens, leaving only alphanumeric words
url_regex = r"([--:\w?@%&+~#=]*\.[a-z]{2,4}\/{0,2})((?:[?&](?:\w+)=(?:\w+))+|[--:\w?@%&+~#=]+)?"
special_character_regex = r"[\"'()[\]]"

comments['body'] = comments['body'].astype('str')

#remove urls, special characters, and replace hyphens with a space
comments['clean'] = comments['body'].apply(lambda text: text.strip().lower()).apply(lambda text: re.sub(url_regex, '', text)).apply(lambda text: re.sub(special_character_regex, '', text)).apply(lambda text: re.sub(r"-", ' ', text))
comments['clean'].head()

<ipython-input-30-993bcaba2f48>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['body'] = comments['body'].astype('str')
<ipython-input-30-993bcaba2f48>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['clean'] = comments['body'].apply(lambda text: text.strip().lower()).apply(lambda text: re.sub(url_regex, '', text)).apply(lambda text: re.sub(special_character_regex, '', text)).apply(lambda text: re.sub(r"-", ' ', text))


0    1 she got to be a bigwig at google by sleeping...
1    for those about to lynch this guy here is a sh...
2    it feels like black people are the most racist...
4    its because otherwise thats all that would app...
5    please go to facebook and comment and post on ...
Name: clean, dtype: object

In [31]:
comments['tokens'] = comments['clean'].apply(lambda text: re.sub(r"[.,!?]"," ", text)).apply(lambda text: re.sub(r"[0-9]", " ", text)).apply(nltk.wordpunct_tokenize)

<ipython-input-31-d6433c9309aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['tokens'] = comments['clean'].apply(lambda text: re.sub(r"[.,!?]"," ", text)).apply(lambda text: re.sub(r"[0-9]", " ", text)).apply(nltk.wordpunct_tokenize)


In [32]:
comments = comments.reset_index(drop=True)
print(len(comments), "total comments")
comments.head()

55567 total comments


,created_utc,subreddit,body,clean,tokens
0,1430438402,news,1-She got to be a bigwig at Google by sleeping...,1 she got to be a bigwig at google by sleeping...,"[she, got, to, be, a, bigwig, at, google, by, ..."
1,1430438407,news,For those about to lynch this guy [here](http:...,for those about to lynch this guy here is a sh...,"[for, those, about, to, lynch, this, guy, here..."
2,1430438439,news,It feels like black people are the most racist...,it feels like black people are the most racist...,"[it, feels, like, black, people, are, the, mos..."
3,1430438449,news,Its because otherwise thats all that would app...,its because otherwise thats all that would app...,"[its, because, otherwise, thats, all, that, wo..."
4,1430438456,news,Please go to Facebook and comment and post on ...,please go to facebook and comment and post on ...,"[please, go, to, facebook, and, comment, and, ..."


## Building the RNN

In [33]:
len(comments['clean'])

55567

In [34]:
import tensorflow as tf
from tensorflow import keras

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [35]:
all_text = ""
for comment in comments['clean'][:5000]:
    all_text += "<" + comment + ">"

In [36]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(all_text)

In [37]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count
print("max id:", max_id, "\ndataset_size:", dataset_size)

max id: 81 
dataset_size: 1087981


In [39]:
[encoded] = np.array(tokenizer.texts_to_sequences([all_text])) - 1

In [40]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [41]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [42]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [43]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [44]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [45]:
dataset = dataset.prefetch(1)

## Model

In [65]:
from datetime import datetime

MODEL_NAME = "RNN_GRU_128"
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
NAME = "{}-{}".format(MODEL_NAME, now)

root_logdir = "tf_logs/fit"
logdir = "{}/{}-{}/".format(root_logdir,MODEL_NAME, now)
print("MODEL NAME:", MODEL_NAME)
print("NOW:", now)
print("NAME:", NAME)
print("ROOT LOGDIR:",root_logdir)
print("LOGDIR:", logdir)
print("")

MODEL NAME: RNN_GRU_128
NOW: 20201209192256
NAME: RNN_GRU_128-20201209192256
ROOT LOGDIR: tf_logs/fit
LOGDIR: tf_logs/fit/RNN_GRU_128-20201209192256/



In [66]:
checkpoint_filepath = 'checkpoints/{}/'.format(NAME)
checkpoint_filepath

'checkpoints/RNN_GRU_128-20201209192256/'

In [67]:
model_callbacks = [
    keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_acc'),
    keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1, write_graph=True)
]

In [68]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
    tf.keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(max_id))
], name=NAME)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
model.summary()

Model: "RNN_GRU_128-20201209192256"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_6 (GRU)                  (None, None, 128)         81024     
_________________________________________________________________
gru_7 (GRU)                  (None, None, 128)         99072     
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 81)          10449     
Total params: 190,545
Trainable params: 190,545
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(dataset, epochs=10, callbacks=model_callbacks)

Epoch 1/10
30597/30597 [==============================] - 439s 14ms/step - loss: 2.8348
Epoch 2/10
16628/30597 [===============>..............] - ETA: 3:18 - loss: 2.7176

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir tf_logs/fit/